In [ ]:
from pathlib import Path
from mri_data import utils
import nibabel as nib
import os
import json
import re
import statistics
import numpy as np

In [3]:
dataroot = Path("/media/smbshare/3Tpioneer_bids")
training_work_home = Path("/home/srs-9/Projects/prl_project/training_work_dirs")
work_dir = training_work_home / "test_train0_swinunet" #! edit this
ensemble_dir = work_dir / "ensemble_output"
datalist_file_src = os.path.join(os.getcwd(), "datalist.json")
datalist_file = work_dir / "datalist.json"

In [16]:
with open(datalist_file, 'r') as f:
    datalist = json.load(f)

inferences = []
labels = []
for item in datalist['testing']:
    label = str(item['label'])
    assert os.path.exists(label)
    labels.append(label)
    subject = re.search(r"(sub-ms\d{4})", label)[1]
    session = re.search(r"(ses-\d{8})", label)[1]
    inf = ensemble_dir / subject / session / "flair.phase.t1_ensemble.nii.gz"
    assert inf.exists()
    inferences.append(str(inf))

In [17]:
lesion_dice_scores = []
prl_dice_scores = []
for lab, inf in zip(labels, inferences):
    lab_data = nib.load(lab).get_fdata()
    inf_data = nib.load(inf).get_fdata()
    lesion_dice_scores.append(utils.dice_score(lab_data, inf_data, seg1_val=1, seg2_val=1))
    prl_dice_scores.append(utils.dice_score(lab_data, inf_data, seg1_val=2, seg2_val=2))

In [23]:
print("lesion dice: {:0.2f}".format(statistics.mean(lesion_dice_scores)))
print("PRL    dice: {:0.2f}".format(statistics.mean(prl_dice_scores)))

lesion dice: 0.82
PRL    dice: 0.47


In [ ]:
def dice_score(seg1, seg2, seg1_val=1, seg2_val=1):
    intersection = np.sum((seg1 == seg1_val) & (seg2 == seg2_val))
    volume_sum = np.sum(seg1 == seg1_val) + np.sum(seg2 == seg2_val)
    if volume_sum == 0:
        # ? Why did I originally make this 1.0? Was there good reason, or mistake?
        # return 1.0
        return None
    return 2.0 * intersection / volume_sum